In [2]:
import pandas as pd
import datetime
import numpy as np

# 读取 excel 表格数据
df_original = pd.read_excel("../data/附件1 监测点A空气质量预报基础数据.xlsx",
                            sheet_name='监测点A逐日污染物浓度实测数据')

# 修改 columns
columns = ['date', 'address', 'SO2', 'NO2', 'PM10', 'PM2.5', 'O3', 'CO']
df_original.columns = columns
df = df_original

# 数据预处理
start_date = str(datetime.datetime.strptime('20200825', '%Y%m%d').date())
end_date = str(datetime.datetime.strptime('20200828', '%Y%m%d').date())

df_selected = df[(df['date'] >= start_date) & (
    df['date'] <= end_date)]   # 根据日期筛选数据
df = df_selected
df = df.iloc[:, 2:]  # 去掉前两列数据

# 用于计算 IAQI
IAQI_DICT = {
    'IAQI': [0, 50, 100, 150, 200, 300, 400, 500],
    'SO2': [0, 50, 150, 475, 800, 1600, 2100, 2620],
    'NO2': [0, 40, 80, 180, 280, 565, 750, 940],
    'PM10': [0, 50, 150, 250, 350, 420, 500, 600],
    'PM2.5': [0, 35, 75, 115, 150, 250, 350, 500],
    'O3': [0, 100, 160, 215, 265, 800],
    'CO': [0, 2, 4, 14, 24, 36, 48, 60],
}
POLLUTANT_KEYS = list(IAQI_DICT.keys())[1:]


def get_aqi_index(key, val):
    """
    获取 aqi 索引，返回污染物浓度在数组中的左右下标
    """

    i = 0
    while i < len(IAQI_DICT[key]):
        if val >= IAQI_DICT[key][i] and val < IAQI_DICT[key][i+1]:
            return i, i+1
        elif IAQI_DICT[key][i] == val:
            return i-1, i
        else:
            i += 1


# 保存结果
aqi_results = np.zeros((df.shape[0]), dtype='float64')
top_pollutant_results = np.array(
    ["" for _ in range(df.shape[0])], dtype='U10')

# 循环计算每一行数据的 AQI
for i in range(df.shape[0]):
    row_data = df.iloc[[i], :]  # 获取当前行数据
    iaqi_list = np.zeros((row_data.shape[1]), dtype='float64')  # 保存 iaqi
    # 循环计算每种污染物对应的 iaqi
    for j in range(row_data.shape[1]):
        key = POLLUTANT_KEYS[j]
        val = row_data.iloc[:, j].values[0]
        # 只计算污染物浓度大于 0 且不超标的 iaqi
        if val > 0 and val <= max(IAQI_DICT[key]):
            left, right = get_aqi_index(key, val)
            iaqi = (IAQI_DICT['IAQI'][right] - IAQI_DICT['IAQI'][left]) / (IAQI_DICT[key][right] - IAQI_DICT[key][left])\
                * (val-IAQI_DICT[key][left]) + IAQI_DICT['IAQI'][left]
            iaqi_list[j] = iaqi
    max_index = np.argmax(iaqi_list)
    # print(iaqi_list)
    top_pollutant_results[i] = (POLLUTANT_KEYS[max_index])
    aqi = iaqi_list.max()  # 计算最大的 iaqi 值作为当前行的 AQI
    aqi_results[i] = aqi
aqi_results = np.around(aqi_results)  # 取整

# 保存数据到 excel 表格
df_out = df

df_out['日期'] = df_selected['date'].dt.strftime('%Y-%m-%d')
df_out['AQI'] = aqi_results
df_out['首要污染物'] = top_pollutant_results

columns = ['日期']
columns.extend(POLLUTANT_KEYS)
columns.extend(['AQI', '首要污染物'])

writer = pd.ExcelWriter('./data/第1问.xlsx')
df_out.to_excel(writer, columns=columns, index=False,
                encoding='utf-8', sheet_name='Sheet1')
writer.save()
writer.close()


[ 8.         15.         27.         15.71428571 60.         12.5       ]
[ 7.         20.         24.         14.28571429 46.         12.5       ]
[  7.          38.75        37.          32.85714286 108.18181818
  15.        ]
[  8.          37.5         47.          47.14285714 137.27272727
  17.5       ]
